<a href="https://colab.research.google.com/github/Kuzay3t/3D_Image_Reconstruction/blob/main/3d_Reconstruction_ghcp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [76]:
!pip install open3d
!pip install torchcdist

ERROR: Could not find a version that satisfies the requirement torchcdist (from versions: none)
ERROR: No matching distribution found for torchcdist


In [77]:
# importing necessacry libaries

import pandas as pd
import zipfile
import os
from google.colab import drive
import zipfile
import open3d as o3d
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import plotly.graph_objects as go
from scipy.optimize import linear_sum_assignment

## Loading Dataset from the google drive

In [78]:
drive.mount('/content/drive', force_remount=True)

KeyboardInterrupt: 

In [ ]:
print(os.listdir('/content/drive/My Drive'))

In [ ]:
zip_path = '/content/drive/My Drive/archive.zip'

In [ ]:
extract_path = '/content/ModelNet10'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [ ]:
print(os.listdir('/content/ModelNet10'))

## Preprocessing the Dataset

In [ ]:
# Setting path to your unzipped dataset
root_dir = '/content/ModelNet10/ModelNet10'
category = 'chair'
model_dir = os.path.join(root_dir, category, 'train')
sample_file = [f for f in os.listdir(model_dir) if f.endswith('.off')][0]
file_path = os.path.join(model_dir, sample_file)

In [ ]:
# Writing the function to load .off file as point cloud
def load_off_as_pointcloud(filename, n_points=1024):
    mesh = o3d.io.read_triangle_mesh(filename)
    pcd = mesh.sample_points_uniformly(number_of_points=n_points)
    return np.asarray(pcd.points)


In [ ]:
clean_points = load_off_as_pointcloud(file_path)

In [ ]:
# adding noise to the dataset

def add_noise(points, std=0.02):
    noise = np.random.normal(scale=std, size=points.shape)
    return points + noise

noisy_points = add_noise(clean_points)

In [ ]:
# denoising the image

class DenoiseNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(3, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 3)
        )
    def forward(self, x):
        return self.mlp(x)

model = DenoiseNet()

In [ ]:
# preparing the data for training
# Convert to torch tensors
noisy_tensor = torch.tensor(noisy_points, dtype=torch.float32)
clean_tensor = torch.tensor(clean_points, dtype=torch.float32)

dataset = [(noisy_tensor, clean_tensor)]

## Training the Model

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

In [ ]:
# training the model

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    pred = model(noisy_tensor)
    loss = loss_fn(pred, clean_tensor)
    loss.backward()
    optimizer.step()
    if (epoch+1) % 20 == 0:
        print(f"Epoch {epoch+1}, Loss: {loss.item():.6f}")

In [ ]:
# Predict denoised points
model.eval()
with torch.no_grad():
    denoised = model(noisy_tensor).numpy()

## Visualisation

In [ ]:
# Visualize using open3d
pcd_noisy = o3d.geometry.PointCloud()
pcd_noisy.points = o3d.utility.Vector3dVector(noisy_points)
pcd_denoised = o3d.geometry.PointCloud()
pcd_denoised.points = o3d.utility.Vector3dVector(denoised)

In [ ]:
def plot_point_cloud(points, title="Point Cloud"):
    x, y, z = points[:,0], points[:,1], points[:,2]
    fig = go.Figure(data=[go.Scatter3d(
        x=x, y=y, z=z,
        mode='markers',
        marker=dict(size=2)
    )])
    fig.update_layout(title=title, margin=dict(l=0, r=0, b=0, t=30))
    fig.show()


In [ ]:
o3d.visualization.draw_geometries([pcd_noisy], window_name='Noisy')
o3d.visualization.draw_geometries([pcd_denoised], window_name='Denoised')

## Model Evaluation

In [ ]:
# defining chamfer distance function
def chamfer_distance(pc1, pc2):
    """
    pc1, pc2: numpy arrays of shape (N, 3)
    Computes symmetric Chamfer Distance between two point clouds.
    """
    pc1 = torch.tensor(pc1, dtype=torch.float32).unsqueeze(0)  # [1, N, 3]
    pc2 = torch.tensor(pc2, dtype=torch.float32).unsqueeze(0)  # [1, M, 3]
    from torch import cdist
    dist1 = cdist(pc1, pc2)  # [1, N, M]
    cd = torch.mean(torch.min(dist1, dim=2)[0]) + torch.mean(torch.min(dist1, dim=1)[0])
    return cd.item()

In [ ]:
cd_noisy = chamfer_distance(noisy_points, clean_points)
cd_denoised = chamfer_distance(denoised, clean_points)
print(f"Chamfer Distance (Noisy→Clean): {cd_noisy:.6f}")
print(f"Chamfer Distance (Denoised→Clean): {cd_denoised:.6f}")

In [ ]:
# defining mean square error function
def mse_point_cloud(pc1, pc2):
    """
    Computes Mean Squared Error (MSE) between two point clouds.
    pc1, pc2: numpy arrays of shape (N, 3)
    """
    return np.mean((pc1 - pc2) ** 2)

In [ ]:
mse = mse_point_cloud(denoised, clean_points)
print(f"Mean Squared Error: {mse:.6f}")

In [ ]:
# defining earth mover's distance
def emd_point_cloud(pc1, pc2):
    """
    Computes Earth Mover's Distance (EMD) between two point clouds using the Hungarian algorithm.
    pc1, pc2: numpy arrays of shape (N, 3)
    """
    cost_matrix = np.linalg.norm(pc1[:, np.newaxis, :] - pc2[np.newaxis, :, :], axis=2)
    row_ind, col_ind = linear_sum_assignment(cost_matrix)
    emd = cost_matrix[row_ind, col_ind].mean()
    return emd

In [ ]:
emd = emd_point_cloud(denoised, clean_points)
print(f"Earth Mover's Distance: {emd:.6f}")